In [ ]:
EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def get_links(input_file_name):
    links = []
    with open(input_file_name, "r") as f:
        for line in f:
            c0, c1 = line.strip().replace('\n', '').split('-')
            links.append((c0, c1))
    return links

In [ ]:
links = get_links(EXAMPLE)
print(links)

In [ ]:
def get_nodes(links):
    nodes = {}
    for c0, c1 in links:
        if c0 in nodes:
            nodes[c0].append(c1)
        else:
            nodes[c0] = [c1]
        if c1 in nodes:
            nodes[c1].append(c0)
        else:
            nodes[c1] = [c0]
    return nodes

In [ ]:
nodes = get_nodes(links)
print(nodes)

In [ ]:
def get_groups(nodes: dict[str, list[str]]):
    groups = set()
    while nodes:
        # We pop to avoid processing the same nodes multiple times
        node0, connected_nodes = nodes.popitem()
        while connected_nodes:
            node1 = connected_nodes.pop()
            if node1 not in nodes:
                continue
            for node2 in connected_nodes:
                # node1 and node2 are both connected to node0
                if node2 in nodes[node1]:
                    # node1 and node2 are connected to each other
                    if node0.startswith('t') or node1.startswith('t') or node2.startswith('t'):
                        # At least one machine starts with 't'
                        groups.add(tuple(sorted([node0, node1, node2])))
    return groups

In [ ]:
print(sorted(get_groups(nodes)))

In [ ]:
def part_1(input_file_name):
    links = get_links(input_file_name)
    nodes = get_nodes(links)
    groups = get_groups(nodes)
    result = len(groups)
    print(result)

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

The text is ambiguous: for part 2, the condition that one computer name must start with 't' isn't applicable anymore!

We start by converting everything to sets, to take advantage of set methods.

In [ ]:
links = get_links(EXAMPLE)
nodes = get_nodes(links)
links = [set(list(link)) for link in links]
for node in nodes:
    nodes[node] = set(nodes[node])
print(nodes)

Given a group of interconnected computers of size n, we can build the groups of size n+1

In [ ]:
def get_bigger_groups_from_group(group, nodes):
    new_nodes = set()
    # Find the nodes that are connected to every node in group
    for node in group:
        if not new_nodes:
            new_nodes = nodes[node]
        else:
            new_nodes = new_nodes.intersection(nodes[node])
    bigger_groups = []
    for new_node in new_nodes:
        # For each node connected to every node in group, create a new group by adding that node to group
        bigger_group = group.union(set([new_node]))
        bigger_groups.append(bigger_group)
    return bigger_groups

Now we can generalize to find all groups of size n+1 given a list of groups of size n

In [ ]:
def get_bigger_groups(groups, nodes):
    # Use a set to guarantee uniqueness
    bigger_groups = set()
    for group in groups:
        bigger_groups_for_group = get_bigger_groups_from_group(group, nodes)
        for bigger_group in bigger_groups_for_group:
            # A set isn't hashable so we convert it to a sorted tuple
            unique_bigger_group = tuple(sorted(list(bigger_group)))
            bigger_groups.add(unique_bigger_group)
    return [set(g) for g in bigger_groups]

Now we start with the pairs of connected computers and build bigger and bigger groups until no more than one remains

In [ ]:
def find_biggest_groups(links, nodes):
    groups = links
    while len(bigger_groups := get_bigger_groups(groups, nodes)) > 1:
        groups = bigger_groups
    return bigger_groups

In [ ]:
biggest_groups = find_biggest_groups(links, nodes)
print(biggest_groups)
print(",".join(sorted(list(biggest_groups[0]))))

In [ ]:
def part_2(input_file_name):
    links = get_links(input_file_name)
    nodes = get_nodes(links)
    links = [set(list(link)) for link in links]
    for node in nodes:
        nodes[node] = set(nodes[node])
    biggest_groups = find_biggest_groups(links, nodes)
    result = ",".join(sorted(list(biggest_groups[0])))
    print(result)

In [ ]:
part_2(EXAMPLE)

In [ ]:
part_2(INPUT)